<h1>New Experiment 2: Random rejection of motion-free subjects</h1>
In this notebook, random rejection tests were conducted using data from S1 in the still condition. The generalisation tests were conducted over S2. Each time we fix the number of retained volumes N. We randomly select N volumes from a subject's data, then we train our proposed model over this sample. At testing, we applied the same rejection scheme to S2, then test over the subset of S2.</br>
We would like to show this proposed model is robust, by which it is independent of 1. The proportion of discarded volumes and 2. The different combinations of b-values.</br>
<h2>Packages</h2>

In [1]:
import nibabel as nib
import numpy as np
import os
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy
"""
package to store the intermediate result
"""
import pickle
"""
package to work out RMSE and SSIM
"""
from utils import calc_RMSE, calc_ssim, load_nii_image, filter_mask

2022-07-16 08:44:22.722220: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


<h2>The class for parsing command line arguments</h2>

In [2]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

<h2>Data Preprocessing</h2>

In [3]:
"""
Using nib to fetch the ground truth img
"""
#load the truth data for subject 1
# s03 data repo
s03_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/'
s03_NDI_path = os.path.join(s03_path, 'corrected_ndi.nii')
s03_ODI_path = os.path.join(s03_path, 'corrected_odi.nii')
s03_FWF_path = os.path.join(s03_path, 'corrected_fwf.nii')

In [4]:
s03_mask_path = os.path.join(s03_path, 'filtered_mask.nii')

In [5]:
s03_mask = load_nii_image(s03_mask_path)

In [6]:
# load the truth data for subject 3
# s03_NDI_img = nib.load(s03_NDI_path)
# s03_ODI_img = nib.load(s03_ODI_path)
# s03_FWF_img = nib.load(s03_FWF_path)
# s03_NDI_affine = s03_NDI_img.affine
# s03_ODI_affine = s03_ODI_img.affine
# s03_FWF_affine = s03_FWF_img.affine
s03_NDI_img_data = load_nii_image(s03_NDI_path, s03_mask)
s03_ODI_img_data = load_nii_image(s03_ODI_path, s03_mask)
s03_FWF_img_data = load_nii_image(s03_FWF_path, s03_mask)

mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374


------------------------
<h2>Random Rejection Scheme Setup</h2>
Let N to be the tested number of retained volumes. For N, 100 subsampled schemes were drawn randomly from the full scheme (with the first b=0 volume and at least two different b values always included). Each subsampled schem was used to evaluate both techniques. For both 3D CNN and AMICO, N are 60, 40 and 30. We would give further undersampled scheme to 3D CNN, N are 20, 16 and 12.

In [7]:
# a list contains the N for each tested retained volume
# N = [60, 40]
# random_dict = {60:[], 40:[]}
# random_dict

In [8]:
# Generated the random scheme for each tested volume
# for n in N:
#     # for each retained volume, we create 100 subsamples
#     for i in range(100):
#         ones = np.ones(n) # 1 = retained
#         zeros = np.zeros(96-n)

#         scheme = np.random.choice(np.concatenate([ones,zeros]), 96, replace=False)
#         scheme = ' '.join(map(str, scheme))
#         random_dict[n].append(scheme)

In [9]:
# save the rejection scheme
# random_60_file = 'random_60.pickle'
# random_40_file = 'random_40.pickle'

# with open(random_60_file, 'wb') as handle:
#     pickle.dump(random_dict[60], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_40_file, 'wb') as handle:
#     pickle.dump(random_dict[40], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# If the random rejection is already set, we can load it
random_60 = []
with (open('/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/random_60.pickle', "rb")) as openfile:
    while True:
        try:
            random_60.append(pickle.load(openfile))
        except EOFError:
            break
random_60 = np.array(random_60[0])

random_40 = []
with (open('/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/random_40.pickle', "rb")) as openfile:
    while True:
        try:
            random_40.append(pickle.load(openfile))
        except EOFError:
            break
random_40 = np.array(random_40[0])

random_30 = []
with (open('/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/random_30.pickle', "rb")) as openfile:
    while True:
        try:
            random_30.append(pickle.load(openfile))
        except EOFError:
            break
random_30 = np.array(random_30[0])

In [11]:
amico_rmse_dict = {60:[], 40:[], 30:[]}
amico_ssim_dict = {60:[], 40:[], 30:[]}

----------------------------
<h2>Model fit AMICO</h2>

In [12]:
def writefile(path, file, combine, savename):
    with open(path+savename, 'w') as fout:
        read_path = path+file
        read_file = open(read_path, 'r')
        lines = read_file.readlines()
        for line in lines:
            temp = line.split()
            temp = [e for e, b in zip(temp, combine) if b == 1]
            fout.write(' '.join(e for e in temp))
            fout.write('\n')
    fout.close()

def writediffusion(path, file, combine, savename):
    img = nib.load(path+file)
    data = img.get_fdata()
    data = data[..., combine==1]
    img = nib.Nifti1Image(data, np.eye(4))
    nib.save(img, path+savename)

In [13]:
def run_amico(subject_path, diffusion, bvals, bvecs, suffix):
    # run amico
    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(subject_path+bvals, subject_path+bvecs)
    ae = amico.Evaluation("/home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI", subject_path.split('/')[-2], verbose=0)
    # load the data
    ae.load_data(dwi_filename = diffusion, scheme_filename = bvals+'.scheme', mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels(regenerate=True)
    ae.load_kernels()
    # model fit.
    ae.fit()
    # ICVF = NDI
    # ISOVF = FWF
    # OD = ODI
    ae.save_results(path_suffix=suffix)

----------------------
<h2>N=60</h2>

In [ ]:
for i in range(len(random_60)//2):

    # read the movefile
    movefile = random_60[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '60_volume'
    bvals = '60_bvals'
    bvecs = '60_bvecs'
    diffusion = '60_diffusion'

    writefile(s03_path, 'bvals', combine, savename=bvals)
    writefile(s03_path, 'bvecs', combine, savename=bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, savename=diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = load_nii_image(icvf_path, s03_mask)
    od = load_nii_image(od_path, s03_mask)
    isovf = load_nii_image(isovf_path, s03_mask)

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = np.sqrt(((s03_NDI_img_data-icvf)**2).mean())
    odi_rmse = np.sqrt(((s03_ODI_img_data-od)**2).mean())
    fwf_rmse = np.sqrt(((s03_FWF_img_data-isovf)**2).mean())
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)

In [14]:
save_path = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results/AMICO/'

In [ ]:
with open(save_path+'ami_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_60)//2, len(random_60)):

    # read the movefile
    movefile = random_60[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '60_volume'
    bvals = '60_bvals'
    bvecs = '60_bvecs'
    diffusion = '60_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = load_nii_image(icvf_path, s03_mask)
    od = load_nii_image(od_path, s03_mask)
    isovf = load_nii_image(isovf_path, s03_mask)

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = np.sqrt(((s03_NDI_img_data-icvf)**2).mean())
    odi_rmse = np.sqrt(((s03_ODI_img_data-od)**2).mean())
    fwf_rmse = np.sqrt(((s03_FWF_img_data-isovf)**2).mean())
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)

In [ ]:
with open(save_path+'ami_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

------
<h2>N=40</h2>

In [ ]:
for i in range(len(random_40)//2):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '40_volume'
    bvals = '40_bvals'
    bvecs = '40_bvecs'
    diffusion = '40_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = load_nii_image(icvf_path, s03_mask)
    od = load_nii_image(od_path, s03_mask)
    isovf = load_nii_image(isovf_path, s03_mask)

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = np.sqrt(((s03_NDI_img_data-icvf)**2).mean())
    odi_rmse = np.sqrt(((s03_ODI_img_data-od)**2).mean())
    fwf_rmse = np.sqrt(((s03_FWF_img_data-isovf)**2).mean())
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[40].append(RMSE)
    amico_ssim_dict[40].append(SSIM)

In [ ]:
with open(save_path+'ami_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_40)//2, len(random_40)):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '40_volume'
    bvals = '40_bvals'
    bvecs = '40_bvecs'
    diffusion = '40_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = load_nii_image(icvf_path, s03_mask)
    od = load_nii_image(od_path, s03_mask)
    isovf = load_nii_image(isovf_path, s03_mask)

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = np.sqrt(((s03_NDI_img_data-icvf)**2).mean())
    odi_rmse = np.sqrt(((s03_ODI_img_data-od)**2).mean())
    fwf_rmse = np.sqrt(((s03_FWF_img_data-isovf)**2).mean())
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[40].append(RMSE)
    amico_ssim_dict[40].append(SSIM)

In [ ]:
with open(save_path+'ami_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h1>N=30</h1>

In [15]:
for i in range(len(random_30)//2):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '30_volume'
    bvals = '30_bvals'
    bvecs = '30_bvecs'
    diffusion = '30_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = load_nii_image(icvf_path, s03_mask)
    od = load_nii_image(od_path, s03_mask)
    isovf = load_nii_image(isovf_path, s03_mask)

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = np.sqrt(((s03_NDI_img_data-icvf)**2).mean())
    odi_rmse = np.sqrt(((s03_ODI_img_data-od)**2).mean())
    fwf_rmse = np.sqrt(((s03_FWF_img_data-isovf)**2).mean())
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[30].append(RMSE)
    amico_ssim_dict[30].append(SSIM)


-> Precomputing rotation matrices:


   |██████████████████████████████████████████████████████████| 100.0%

   [ DONE ]
-> Writing scheme file to [ /home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/30_bvals.scheme ]

-> Loading data:
	* DWI signal
		- dim    = 84 x 84 x 50 x 40
		- pixdim = 1.000 x 1.000 x 1.000
	* Acquisition scheme
		- 40 samples, 3 shells
		- 3 @ b=0 , 12 @ b=2000.0 , 8 @ b=1000.0 , 17 @ b=3000.0 
	* Binary mask
		- dim    = 84 x 84 x 50
		- pixdim = 2.500 x 2.500 x 2.500
		- voxels = 90060
   [ 0.0 seconds ]

-> Preprocessing:
	* Normalizing to b0... [ min=0.00,  mean=0.66, max=12077.38 ]
	* Keeping all b0 volume(s)
   [ 0.1 seconds ]

-> Creating LUT for "NODDI" model:



   |██████████████████████████████████████████████████████████| 100.0%


   [ 133.7 seconds ]

-> Resampling LUT for subject "s03_still_reg":


...

   [ 138.7 seconds ]

-> Resampling LUT for subject "s03_still_reg":


   |██████████████████████████████████████████████████████████| 100.0%


   [ 41.7 seconds ]

-> Fitting "NODDI" model to 90060 voxels:


   |██████████████████████████████████████████████████████████| 100.0%


   [ 00h 00m 28s ]

-> Saving output to "AMICO/NODDI_30_volume/*":
	- configuration  [OK]
	- FIT_dir.nii.gz  [OK]
	- FIT_ICVF.nii.gz  [OK]
	- FIT_OD.nii.gz  [OK]
	- FIT_ISOVF.nii.gz  [OK]
   [ DONE ]
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374


In [16]:
with open(save_path+'ami_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
for i in range(len(random_30)//2, len(random_30)):

    # read the movefile
    movefile = random_30[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    suffix = '30_volume'
    bvals = '30_bvals'
    bvecs = '30_bvecs'
    diffusion = '30_diffusion'

    writefile(s03_path, 'bvals', combine, bvals)
    writefile(s03_path, 'bvecs', combine, bvecs)
    writediffusion(s03_path, 'diffusion.nii', combine, diffusion)

    run_amico(s03_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)

    # analyse the result
    icvf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz')
    isovf_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz')
    od_path = os.path.join(s03_path, 'AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz')

    icvf = load_nii_image(icvf_path, s03_mask)
    od = load_nii_image(od_path, s03_mask)
    isovf = load_nii_image(isovf_path, s03_mask)

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = np.sqrt(((s03_NDI_img_data-icvf)**2).mean())
    odi_rmse = np.sqrt(((s03_ODI_img_data-od)**2).mean())
    fwf_rmse = np.sqrt(((s03_FWF_img_data-isovf)**2).mean())
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[30].append(RMSE)
    amico_ssim_dict[30].append(SSIM)


-> Precomputing rotation matrices:


   |██████████████████████████████████████████████████████████| 100.0%

   [ DONE ]
-> Writing scheme file to [ /home/vw/Data/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/30_bvals.scheme ]

-> Loading data:
	* DWI signal
		- dim    = 84 x 84 x 50 x 30
		- pixdim = 1.000 x 1.000 x 1.000
	* Acquisition scheme
		- 30 samples, 3 shells
		- 3 @ b=0 , 10 @ b=3000.0 , 11 @ b=2000.0 , 6 @ b=1000.0 
	* Binary mask
		- dim    = 84 x 84 x 50
		- pixdim = 2.500 x 2.500 x 2.500
		- voxels = 90060
   [ 0.0 seconds ]

-> Preprocessing:
	* Normalizing to b0... [ min=0.00,  mean=0.67, max=2989.29 ]
	* Keeping all b0 volume(s)
   [ 0.0 seconds ]

-> Creating LUT for "NODDI" model:



   |██████████████████████████████████████████████████████████| 100.0%


   [ 139.3 seconds ]

-> Resampling LUT for subject "s03_still_reg":


...

   [ 136.5 seconds ]

-> Resampling LUT for subject "s03_still_reg":


   |██████████████████████████████████████████████████████████| 100.0%


   [ 20.4 seconds ]

-> Fitting "NODDI" model to 90060 voxels:


   |██████████████████████████████████████████████████████████| 100.0%


   [ 00h 00m 18s ]

-> Saving output to "AMICO/NODDI_30_volume/*":
	- configuration  [OK]
	- FIT_dir.nii.gz  [OK]
	- FIT_ICVF.nii.gz  [OK]
	- FIT_OD.nii.gz  [OK]
	- FIT_ISOVF.nii.gz  [OK]
   [ DONE ]
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374
mask has shape: (352800,)
data befor masking has shape: (352800, 1)
data after masking has shape: (90060, 1) the ratio of masked voxel is: 0.2552721088435374


In [18]:
with open(save_path+'ami_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+'ami_ssim_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_path+'ami_ssim_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)